In [ ]:
# default_exp examine
%load_ext autoreload
%autoreload 2

# examine


> API details.

In [ ]:
#hide
from nbdev.showdoc import *

![pipeline](images/graphic11.PNG)

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_windows.ipynb.
Converted 02_explore.ipynb.
Converted 03_data.ipynb.
Converted 03_preprocessing.ipynb.
Converted 04_pipeline.ipynb.
Converted 05_train.ipynb.
Converted 06_examine.ipynb.
Converted 90_tutorial.ipynb.
Converted index.ipynb.
